In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(dotenv_path='.env')

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [3]:
import os
from dotenv import load_dotenv
from crewai.tools import BaseTool
from pydantic import Field
from langchain_community.utilities import GoogleSerperAPIWrapper

# Set up your SERPER_API_KEY key in an .env file, eg:
# SERPER_API_KEY=<your api key>
load_dotenv()

search = TavilySearchResults()

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: TavilySearchResults = Field(default_factory=TavilySearchResults)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

# rest of the code ...


In [4]:

# Agent 1: Industry Researcher
from crewai import Agent
from crewai_tools import WebsiteSearchTool,FileWriterTool,DirectoryReadTool,FileReadTool
researcher = Agent(
    role="Industry Researcher",
    goal=(
        "Conduct comprehensive market research for the target company. "
        "Identify the industry's segmentation, key offerings, vision, and strategic focus areas. "
        "Analyze competitors' strategies and gather insights from annual reports."
    ),
    backstory=(
        "An experienced business analyst with over a decade of expertise in extracting key market insights. "
        "You specialize in identifying value propositions and competitive edges."
    ),
    tools=[SearchTool(), FileWriterTool()],
    verbose=True,
    memory=True,
    llm="gpt-4o-mini",
    allow_delegation=True,
)

# Agent 2: AI Use Case Specialist
use_case_generator = Agent(
    role="AI Use Case Specialist",
    goal=(
        "Analyze market research findings and propose actionable AI/ML use cases "
        "to enhance operations, customer satisfaction, and efficiency."
    ),
    backstory=(
        "A domain expert in AI applications, known for creating impactful solutions for industry challenges. "
        "You align technology capabilities with business needs."
    ),
    tools=[SearchTool(), FileWriterTool()],
    verbose=True,
    memory=True,
    llm="gpt-4o-mini",
    allow_delegation=True,
)

# Agent 3: Content Writer
writer = Agent(
    role="Content Writer",
    goal=(
        "Synthesize findings into an engaging and comprehensive market research report, "
        "highlighting industry insights, AI/ML opportunities, and strategic recommendations."
    ),
    backstory=(
        "A skilled writer with a passion for simplifying complex technical analyses into clear, actionable insights. "
        "You excel at crafting content that resonates with technical and non-technical audiences alike."
    ),
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],
    verbose=True,
    memory=True,
    llm="gpt-4o-mini",
    allow_delegation=True,
)

In [5]:

# Task Definitions
from crewai import Task
# Task 1: Industry Research
industry_research_task = Task(
    description="Conduct comprehensive market research for {company}. Include a deep dive into industry trends, competitor strategies, financial benchmarks, and consumer behavior insights. Highlight technological adoption and areas for improvement.",
    expected_output="A report summarizing key findings, competitor analysis, industry benchmarks, and technological trends."
                     "Annual reports of competetor should also be analysed "
                     "Gather all the relavent links",
    agent=researcher,
)


# Task 2: AI Use Case Generation
use_case_task = Task(
    description="Based on market research, propose innovative and practical AI/ML use cases for {company}. Provide relevance to their strategic goals, datasets required, and feasibility analysis.",
    expected_output="A detailed document listing creative AI/ML use cases with feasibility assessments and potential datasets."
                     "Should dataset link",
    agent=use_case_generator,
)

# Task 3: Write Market Research Report
write_report_task = Task(
    description="Write a comprehensive market research report including industry analysis, competitor benchmarks, proposed AI/ML use cases, and strategic recommendations. Each section should provide actionable insights and measurable outcomes.",
    expected_output="A professional report covering in-depth market research, competitor analysis, AI/ML use cases, and actionable recommendations. Include data visualizations and references."
    "The report should be well detailed and should be covering all the usecases."
    "Add relavent links for annual report usecase dataset form platfroms like github,huggingface and kaggle",
    agent=writer,
)


In [6]:
from crewai import Crew
crew = Crew(
    agents=[researcher,use_case_generator,writer],
    tasks=[industry_research_task, use_case_task,write_report_task],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
output = crew.kickoff(inputs={'company':'SpaceX'})

 
[2024-12-27 21:31:52][INFO]: Planning the crew execution
# Agent: Industry Researcher
## Task: Conduct comprehensive market research for SpaceX. Include a deep dive into industry trends, competitor strategies, financial benchmarks, and consumer behavior insights. Highlight technological adoption and areas for improvement.1. Start with a clear understanding of SpaceX's current market position and focus areas.  
2. Utilize the Search Tool to conduct a wide-ranging search for industry trends, competitor strategies, financial benchmarks, and consumer behavior insights. Specifically, use keywords like 'SpaceX market research', 'aerospace industry trends', 'SpaceX competitor analysis', 'financial performance of SpaceX vs competitors', and 'consumer insights in aerospace'.  
3. Collect all relevant findings and data from the search results, paying particular attention to annual reports of competitors to understand their strategic focuses.  
4. Synthesize the insights gathered into a structu

In [11]:
from Crew import generator

output = generator('Tesla')

 
[2024-12-27 22:00:57][INFO]: Planning the crew execution
# Agent: Industry Researcher
## Task: Conduct comprehensive market research for Tesla. Include a deep dive into industry trends, competitor strategies, financial benchmarks, and consumer behavior insights. Highlight technological adoption and areas for improvement.1. Initiate the market research by using the Search tool to identify the latest industry trends related to electric vehicles and Tesla. 
2. Conduct multiple searches focusing on competitor strategies including major players like Ford, GM, and others in the EV space. 
3. Analyze the financial benchmarks of Tesla and its competitors by searching for their annual reports; extract critical financial metrics such as revenue, profit margins, and market share. 
4. Investigate consumer behavior insights through searching for studies or reports regarding Tesla customers and general EV consumer psychology. 
5. Highlight areas where technology adoption is lagging within Tesla an

2024-12-27 22:01:28,193 - 2472 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "d:\researcherai\venv\lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
  File "d:\researcherai\venv\lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "d:\researcherai\venv\lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "d:\researcherai\venv\lib\http\client.py", line 1368, in getresponse
    response.begin()
  File "d:\researcherai\venv\lib\http\client.py", line 317, in begin
    version, status, reason = self._read_status()
  File "d:\researcherai\venv\lib\http\client.py", line 286, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handli

In [15]:
from markdown import markdown
print(markdown(output.raw))

<h1>Comprehensive Market Research Report: Tesla's AI/ML Opportunities and Strategic Recommendations</h1>
<h2>1. Industry Analysis</h2>
<p>The automotive industry is undergoing a transformative phase, particularly with the rise of electric vehicles (EVs). Market conditions have shifted dramatically in favor of sustainable technologies, reflecting a surge in consumer demand for environmentally friendly transportation. In recent years, technology adoption trends have indicated that automakers are increasingly integrating advanced technologies such as Artificial Intelligence (AI) and Machine Learning (ML) to enhance operational efficiency and improve user experiences.</p>
<p>The significance of innovation in the automotive domain cannot be overstated, especially concerning EVs. Companies that lead in AI/ML integration are positioning themselves ahead of competitors by proactively addressing challenges such as energy management, vehicle connectivity, and user safety. Tesla, as a pioneer in 

In [7]:
with open('reports/detailed_use_cases.md','w') as file:
    file.write(output.raw)

FileNotFoundError: [Errno 2] No such file or directory: 'reports/detailed_use_cases.md'

In [10]:
print(output)

## Comprehensive Market Research Report: SpaceX

### 1. Industry Analysis
The aerospace industry is rapidly evolving with advancements in technology and increasing investments in private space exploration. Companies like SpaceX are at the forefront, leveraging cutting-edge technologies such as artificial intelligence and machine learning to enhance operational efficiencies and reduce costs.

The market analysis indicates a growing demand for space exploration, satellite services, and defense applications, driven by both commercial and governmental initiatives.

### 2. Competitor Benchmarks
Key competitors in the market include Blue Origin, Boeing, and Lockheed Martin. These companies are also investing in AI and automation to optimize their operations, improve safety, and reduce costs.

Benchmarking SpaceX against these companies reveals its leadership in reusability of rocket components and rapid launch capabilities, which provide it with a competitive edge.

### 3. Proposed AI/ML Use

In [8]:
import streamlit as st
st.markdown(output.raw)

2024-12-27 21:37:57.117 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 21:37:57.608 
  command:

    streamlit run d:\researcherai\venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-27 21:37:57.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [28]:
!pip install weasyprint

In [29]:
from markdown import markdown
from weasyprint import HTML, CSS
from io import BytesIO

def convert_markdown_to_pdf_memory(markdown_text):
    """
    Converts Markdown text to PDF in memory using Python.

    Args:
        markdown_text (str): The Markdown text to convert.

    Returns:
        bytes: The PDF content as bytes.
    """

    html = markdown(markdown_text)

    # Apply CSS styling (optional)
    css = CSS(string='''
        body {
            font-family: sans-serif;
            margin: 1em;
        }
        h1 {
            font-size: 2em;
        }
    ''')

    # Generate PDF in memory
    pdf_buffer = BytesIO()
    HTML(string=html).write_pdf(pdf_buffer, stylesheets=[css])
    pdf_buffer.seek(0)
    pdf_content = pdf_buffer.read()

    return pdf_content

# Example usage
markdown_text = """
# This is a Markdown heading

This is some sample text.

* Item 1
* Item 2
"""

pdf_content = convert_markdown_to_pdf_memory(markdown_text)

# Do something with the PDF content, 
# e.g., save it to a file, display it, or send it as an email attachment


-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'libgobject-2.0-0': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0'

In [62]:
import pdfkit
from markdown import markdown
def convert_md_to_pdf(md_file, pdf_file):
    path_to_wkhtml = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"

    config = pdfkit.configuration(wkhtmltopdf= path_to_wkhtml)
    with open(md_file, 'r', encoding='utf-8') as file:
        text = file.read()
        
        html_content = markdown(text)
        pdfkit.from_string(html_content,pdf_file,configuration= config)

md_file_path = "reports/market_research_report.md"
pdf_file_path = "reports/market_research_report.pdf"
convert_md_to_pdf(md_file_path, pdf_file_path)

In [63]:
convert_md_to_pdf('research_reports/industry_summary.md','research_reports/industry_summary.pdf')

True

In [ ]:
def md_to_pdf